<a href="https://colab.research.google.com/github/AndriiKulakovskyi/image_processing/blob/main/ai_isp_filters/denoising/noise2noise/n2n_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision torchaudio

     |████████████████████████████████| 1.9MB 17.0MB/s 


In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random input and output data
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# Randomly initialize weights
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)

    # Compute and print loss
    loss = np.square(y_pred - y).sum()
    print(t, loss)

    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)

    # Update weights
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

0 32186665.077434506
1 32454842.36710629
2 36386739.79842627
3 37438569.52340772
4 31571761.36067578
5 20349086.78603634
6 10611095.266302504
7 5056530.689777068
8 2610075.4515889157
9 1583096.903594486
10 1119180.972784773
11 872416.4032448514
12 716539.7167165542
13 604265.7567556651
14 517029.97007657104
15 446478.4447441645
16 388210.0750232955
17 339432.9271095489
18 298174.57355928526
19 263036.6515918931
20 232945.90620689056
21 206998.18648234894
22 184514.45769920386
23 164927.6550709013
24 147812.1535087924
25 132791.88145135986
26 119574.44049752844
27 107903.05306243367
28 97574.63466339692
29 88410.3147839673
30 80264.54189844475
31 72998.24146361853
32 66498.24869675997
33 60670.31890234611
34 55432.70589335048
35 50717.551781665454
36 46466.14676098227
37 42636.243120795436
38 39169.063733522824
39 36026.28118300586
40 33173.45355959948
41 30578.824988643555
42 28216.70688295624
43 26062.177947972777
44 24094.161313166107
45 22296.131570776968
46 20650.633390064446
47 19